In [2]:
# !pip uninstall numpy -y
# !pip install numpy
from varz import Vars
import GPy
import tensorflow as tf
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from sklearn.cluster import KMeans

# Importing models
from NSGPy.stheno.model import NSGPRegression as NSS
from NSGPy.torch.model import NSGPRegression as NST

### Common data and params

In [3]:
seed = 0
input_dim = 10
num_inducing_points = 15
N = 100

np.random.seed(seed)
rand = lambda shape: np.abs(np.random.normal(loc=0, scale=1, size=shape))
local_std = rand((input_dim,))
local_gp_ls = rand((input_dim,))
local_ls = rand((input_dim, num_inducing_points))
local_gp_noise_std = rand((input_dim,))

global_std = np.random.normal()
global_gp_noise_std = np.random.normal()

f_indu = lambda x, num_ind: KMeans(n_clusters=num_ind, random_state=seed).fit(x).cluster_centers_

X = np.random.rand(N,input_dim)
y = np.random.rand(N,1)

X_test = np.random.rand(N*2, input_dim)

### GPy

In [ ]:
m = GPy.models.GPRegression(X, y, GPy.kern.RBF(input_dim, ARD=True))
GPy_pred_y, GPy_pred_var = m.predict(X_test, full_cov=True)

### Stheno model

In [ ]:
vs = Vars(tf.float64)
# Local params
vs.positive(init=local_std, shape=(input_dim,), name='local_std')
vs.positive(init=local_gp_ls, shape=(input_dim,), name='local_gp_ls')
vs.positive(init=local_ls, 
                    shape=(input_dim, num_inducing_points), name='local_ls')
vs.positive(init=local_gp_noise_std, shape=(input_dim,), name='local_gp_noise_std')

# Global params
vs.positive(init=global_std, name='global_std')
vs.positive(init=global_gp_noise_std, name='global_gp_noise_std')

model = NSS(X, y, vs, num_inducing_points, f_indu, seed=seed)

mean_y, var_y = model.predict(X_test)
mean_y

In [ ]:
torch.manual_seed(seed)
f = lambda size: (1+torch.rand(size)).requires_grad_()
self.params['local_gp_std'] = f((self.input_dim,))
self.params['local_gp_ls'] = f((self.input_dim,))
self.params['local_gp_noise_std'] = f((self.input_dim,))
self.params['local_ls'] = f((self.num_inducing_points, self.input_dim))
self.params['global_gp_std'] = f((1,))
self.params['global_gp_noise_std'] = f((1,))

### Appendix

In [4]:
from stheno import GP, EQ

In [42]:
f = GP(EQ())
f_ = f | (f(X[:,0], 0.5), y)
ans1 = f_(X_test[:,0]).mean.mat

In [68]:
def LocalKernel(x1,x2): # return local kernel without variance
    x1 = torch.tensor(x1.reshape(-1,1))
    x2 = torch.tensor(x2.reshape(-1,1))
    
    dist = x1 - x2.T
    scaled_dist = dist

    return torch.exp(-scaled_dist**2/2)

In [69]:
k1 = EQ()(X[:,0:1]).mat
k2 = LocalKernel(X[:,0:1], X[:,0:1])

k1[:2,:2]

array([[1.        , 0.98187768],
       [0.98187768, 1.        ]])

In [70]:
k2[:2,:2]

tensor([[1.0000, 0.9819],
        [0.9819, 1.0000]])

In [46]:
K = LocalKernel(X[:,0], X[:,0])
K += np.eye(X.shape[0])*0.5
K_ = LocalKernel(X_test[:,0], X[:,0])

L = torch.linalg.cholesky(torch.tensor(K))

alpha = torch.cholesky_solve(torch.tensor(y), L)
ans2 = torch.tensor(K_)@alpha

 <ipython-input-46-f461e699de8e>:5: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 <ipython-input-46-f461e699de8e>:8: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [47]:
np.allclose(ans1, ans2)

False

In [48]:
ans1[:3], ans2[:3].numpy()

(array([[0.54375254],
        [0.52426359],
        [0.49439437]]),
 array([[0.54402733],
        [0.53669224],
        [0.52267955]]))